Demonstrate we can specify a simple interface to load data sources.

In [1]:
import numpy as np
import pandas as pd

%load_ext autoreload

In [2]:
%autoreload 2

from ldr import Schema, Filter

In [3]:
sources = { "fx": [ "tests/media/gbp-usd.csv", Schema(desc = "fx") ], 
           "sales": [ "tests/media/sales.csv", Schema(desc = "sales") ],  
           "weather": [ "tests/media/london.csv", Schema(desc = "weather") ] }

In [4]:
filters = list(map( lambda x: Filter(x[0], x[1]), sources.values()))

In [5]:
def postprocess0(f0):
    f0.series("datetime", index=True)
    return f0.series("value")

In [6]:
series = list(map(lambda x: postprocess0(x), filters))

Finally, merge the series together

In [66]:
df1 = pd.DataFrame(series).transpose()
df1.describe()

,fx,sales,weather
count,788.000000,1864.000000,1842.000000
mean,1.366371,1115.015558,12.075244
std,0.098996,927.505853,5.429419
min,1.204800,0.000000,-4.100000
25%,1.292975,310.000000,7.900000
50%,1.338350,852.500000,11.950000
75%,1.437975,1805.250000,16.400000
max,1.577600,4864.000000,27.900000


In [70]:
", ".join(df1.columns.values)

'fx, sales, weather'